In [1]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network 
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
import random
from scipy.stats import beta
import pandas as pd
import copy
%matplotlib inline
%run pure_strategy_selection.ipynb  #include simple selection algorithm
import scipy.io
import collections
import sys


## Mathmatic Functions

In [2]:
# centers the opinion vector around 0\n",
def mean_center(op, n):
    ones = np.ones((n, 1))
    x = op - (np.dot(np.transpose(op),ones)/n) * ones
    return x
    
# compute number of edges, m\n
def num_edges(L, n):
    m = 0
    for i in range(n):
        for j in range(n):
            if i > j and L[i,j] < 0:
                m += 1            
    return m

# maximizing polarization only: \\bar{z}^T \\bar{z}   
def obj_polarization(A, L, op, n):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean) 
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

def obj_polarization_1(A, L, op, n):
    z = np.dot(A, op) 
    z_mean = mean_center(z, n)
    return np.dot(np.transpose(z_mean), z_mean)[0,0] 

# Calculate innate polarization
def obj_innate_polarization(s, n):  
#     np.set_printoptions(precision=5)
    op_mean = mean_center(s, n)
    return np.dot(np.transpose(op_mean), op_mean)[0,0]

In [3]:
# Parameters for the network
# n = 6   # number of nodes in the network
# r = 1/4  # percent of info souce in the network
# n1 = int(n*(1-r))
n = 3
np.set_printoptions(precision=3)

## Creating Network


### 1. Make Exteme Fixed Network

In [5]:

# # Create extrem network 1
# n =3
# c1 = np.sort(np.random.choice(n, n, replace=False)) #assume (1-r) are individuals
# # print('c1')
# # print(c1)
# l1 = len(c1)

# a = np.array([0,  0.5, 0.5])
# b = np.array([0.5,0,   0.5])
# c = np.array([0.5,0.5, 0])

# G = np.vstack((a, b, c))
# # print(G)

# L = scipy.sparse.csgraph.laplacian(G, normed=False)
# A = np.linalg.inv(np.identity(n) + L)
# m = num_edges(L, n)
# columnsum_ij = np.sum(A, axis=0)
# # print('Column Sum')
# # print(columnsum_ij)
# # what the twitter graph looks like 
# # nxG = nx.from_numpy_matrix(G)
# # plt.figure(figsize=(20, 20))
# # nx.draw(nxG)

# La = scipy.sparse.csgraph.laplacian(G, normed=False)

# nxG = nx.from_numpy_matrix(G)

# color_map = []
# for node in nxG:
#     if node in c1:
#         color_map.append('Blue')
#     else: 
#         color_map.append('Red')  

# #nxG1 = nx.DiGraph(G)
# mapping = {0: "0.5", 1: "1", 2: "0"}
# nxG = nx.relabel_nodes(nxG, mapping)
# sorted(nxG)
# #nx.draw(nxG, node_color=color_map,with_labels=True,font_size=12,font_color='w')
# plt.figure(figsize=(6, 6))

# nx.draw(nxG, node_size=2000, with_labels=True,font_size=19, font_color="w", font_weight="bold", width=4, edge_color="grey",cmap=plt.cm.Blues,pos=nx.spectral_layout(nxG))
# plt.show()


### 4. Equilibrium & Polarization  - based on derivation
$$P(z) = z ^T * z $$


In [6]:
# s = make_innat_opinions(n, c1)
# op = s
# print(s)
n = 3
s_0=[1]
s_1=[0.5]
s_2=[0]
s = np.vstack((s_0,s_1,s_2))

# s =  make_innat_opinions(n, c1)
print('Innate Opinion')
print(s)
print('Equilibrium Opinion')
print(np.dot(A, s))
#s = (1,0.5,0) # polarization = 0.75
# s = (1,0,0.5) # polarization = 0.75
# s =(0,1,0.5) # polarization = 0.75
#s = (0.5,1,0) # polarization = 0
op = s
y = mean_center(s,n)
# print(y)
innat_pol = np.dot(np.transpose(y), y)[0,0] 
print('Innate_polarization:')
print(innat_pol)

# Test equilibrium polarization
equ_pol = obj_polarization(A, L, op, n)
print('Equi_polarization:')
print(equ_pol)

di = equ_pol-innat_pol
print("Difference:")
print(di)

Innate Opinion
[[1. ]
 [0.5]
 [0. ]]
Equilibrium Opinion


NameError: name 'A' is not defined

### Testing players' behavior

In [895]:
def random_play(s,n):  # player randomly choose an agent and randomly change the agent
    
    op = copy.copy(s)
  
    v = random.randint(0,n-1)  # randomly select an agent index
#     print(v)
    new_op = random.randint(0, 1)  # randomly select an opininon between 0 and 1
#     print(new_op)
    
    # Store old opinion
    old_opinion = op[v,0]
    
    #update the opinion
    op[v,0] = new_op 
#     print('Only 1 opinion changed')
#     print(op)
    print("    "+"Agent" + str(v) +" 's opinion " + str(old_opinion) + " changed to "+ str(new_op))
    por = obj_polarization(A, L, op, n)
    
    #restore op op array to innate opinion
    op[v] = old_opinion
    print("Network reaches equilibrium Polarization: " + str(por))
#     print('Should be restored')
#     print(op)
    return (v, new_op, por)





In [1007]:
def random_play1(s,n):  # player randomly choose an agent and randomly change the agent
    
    op = copy.copy(s)
#     max_opi_option = random.uniform(0, 1)   # options that maximizer have
    
    #v = random.randint(0,n-1)  # randomly select an agent index
#     print(v)
    v = 1
    #new_op = random.uniform(0, 1)  # randomly select an opininon between 0 and 1
    new_op = 0
#     print(new_op)
    
    # Store old opinion
    old_opinion = op[v,0]
    
    #update the opinion
    op[v,0] = new_op 
#     print('Only 1 opinion changed')
#     print(op)
    print("    "+"Agent" + str(v) +" 's opinion " + str(old_opinion) + " changed to "+ str(new_op))
    por = obj_polarization(A, L, op, n)
    
    #restore op op array to innate opinion
    op[v] = old_opinion
    print("Network reaches equilibrium Polarization: " + str(por))
#     print('Should be restored')
#     print(op)
    return (v, new_op, por)





In [1008]:
## Testing to see if random_play works -- NO NEED TO RUN
# min_touched =[]
# (v1, maxmize_op, innat_equi_por, max_por) = choose_max_vertex(s, n, min_touched)
# print(v1, maxmize_op, innat_equi_por, max_por)

In [1009]:
## Testing to see if random_play works -- NO NEED TO RUN
# (v1, max_opinion, max_pol) = random_play(s,n)
# (v2, min_opinion, min_pol) = random_play(s,n)

In [1010]:
def maximizer_fir_play(s,n,min_touched):    # maxmizer first-time play, greedy algorithm
    op = copy.copy(s)

    print('Maximizer Play')

    max_champion = choose_max_vertex(op, n, min_touched) # The best choice among all opinions and vertexs, function is in "pure_strategy_selection.ipynb"
    (v1, max_opinion, innate_obj, max_pol) = max_champion # find agent v1, and max_opinion that can maxmize the equi_polarization(max_pol)

    if v1 == None:   # if maximizer cannot find one
        print('Maximizer fail')

    else:
        print("                                ")
        print("Maximizer finds its target agent:")
#         print('v1', 'changed_opinion', 'innate_obj', 'obj')
#         print(max_champion)

        #Store innate_op of the max_selected vertex
        old_opinion_max = op[v1, 0]
        ##### change the agent's opinion with best action(agent v1, max_op)
        op[v1,0] = max_opinion
        ## check if agent's opinionis is changed or not
        print("    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
        print("Network reaches equilibrium Polarization: " + str(max_pol))


    return(v1, max_opinion, max_pol)

In [1011]:
max_touched = []
min_touched = []
(v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)
print(v1, max_opinion, max_pol)

Maximizer Play
Max start with agent 0
change op to 0.0
change op to 0.1
Max start with agent 1
change op to 0.0
change op to 0.1
Max start with agent 2
change op to 0.0
change op to 0.1
                                
Maximizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0
Network reaches equilibrium Polarization: 0.5933309600094931
1 0 0.5933309600094931


In [1012]:
##### minimizer first-time play, greedy algorithm
def minimizer_fir_play(s,n,max_touched): 
    
    op = copy.copy(s)
    print('_______________________')
    print('Minimizer Play')
#     print('Only 1 opinion changed')
#     print(op)
    
    min_champion = choose_min_vertex(op, n, max_touched)
    (v2, min_opinion, innat_equi_por, min_pol) = min_champion
    
   #Store innate_op of the min_selected vertex
    old_opinion_min = op[v2,0]
    
    if v2 == None:
        print('Minimizer fail')

    else:
        print("                                ")
        print("Minimizer finds its target agent:")

        ##### change the agent's opinion
        op[v2,0] = min_opinion   #-------------------------------------------------> store minimize strategy


        print("    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))

        print("Network reaches equilibrium Polarization: " + str(min_pol))
#         print('2 opinion changed')
#         print(op)

    return (v2,min_opinion, min_pol)

In [1013]:
max_touched = []
min_touched = []
(v2, min_opinion, min_pol) = minimizer_fir_play(s,n,max_touched)
print(v2, min_opinion, min_pol)

_______________________
Minimizer Play
Min start with agent 0
Min start with agent 1
Min start with agent 2
                                
Minimizer finds its target agent:
    Agent2 's opinion 0.0 changed to 0.75
Network reaches equilibrium Polarization: 0.11124955500177997
2 0.75 0.11124955500177997


In [1014]:
## Testing above functions
# min_touched=[]
# max_touched=[]
# # Game start from maximizer random play
# print('Maximizer random selection')
# (v1, max_opinion, max_pol) = random_play(s,n)
# max_touched.append(v1)
# # print('v1, max_opinion, max_pol')
# # print(v1, max_opinion, max_pol)
# # store maximizer play history, using agent(row) and changed opinion(column) as indicator to locate history




# # Game start from minimizer random play 
# print('Minimizer random selection')
# (v2, min_opinion, min_pol) = minimizer_fir_play(s,n,max_touched)
# min_touched.append(v2)



In [1015]:

# # # # Testing block to see if player operate properly
# (v1, max_opinion, max_pol) = maximizer_play(s,v2,min_opinion,n,min_touched)
# (v2, min_opinion, min_pol) = minimizer_play(s,v1,max_opinion,n, max_touched)
# # result =  minimizer_play(max_input, n)


In [1016]:
# Row are Column are depended on min and max's choice: agent v and opinion 
def row_index(v2, min_opinion):
    row = 11*v2 + min_opinion*10 
    return int(row)
def column_index(v1,max_opinion):
    column = 2*v1 + max_opinion
    return int(column)  #the python dataframe index

### Mixed Strategy Payoff


In [1017]:
def make_payoff_row(op1,v2):
    payoff_row = np.zeros(2*n)
    v1 = 0
#     print('one opinion changed -min')
#     print(op1)
    for column in range(2*n):
#         print(column)
        v1 = int(column/2)  #i.e., column 11 is agent 5, opinion 1
        max_opinion = column%2
#         print(v1, max_opinion)
        # update the maximizer's change to the opinion array that has changed by minimizer(op1)
        op2 = copy.copy(op1)
        op2[v1,0] = max_opinion
#         print('max_opinion')
#         print(v1, max_opinion)
#         print('two opinion changed -min +  max')
#         print(op2)
        # calculate the polarization with both max and min's action
        payoff_row[column] = obj_polarization(A, L, op2, n)
#         print(op2,payoff_row[column])
    # when v1 == v2, the polarization should be negative for max, infinet for min. 
    # Replace the the column_index of agent v2 with 0 for max
    j_1 = 2*v2 + 0
    j_2 = 2*v2 + 1
    payoff_row[j_1] = -100
    payoff_row[j_2] = -100
    
    return payoff_row

In [744]:
# #(1,0) (2,0.3928571428571428)
op1=copy.copy(s)
# print(op1)
op1[2,0] = 1  #op1 is the opinion array that updated by minimizer
# print(op1)
payoff_row_1 = make_payoff_row(op1,2)
print(payoff_row_1)

[   0.445    0.148    0.593    0.    -100.    -100.   ]


### Minimizer Mixed Strategy

In [750]:
#NEEDDDDDDD UPDAE

# Calculate polarization of minimizer's Mixed Strategy
def mixed_min_polarization(s,v2,min_opinion,fla_max_fre):

    op1 =  copy.copy(s) # make a copy of the innate opinion array 
    op1[v2,0] = min_opinion # then only updated by minimizer's current change

    # calculate the polarization with both min(did here) and max's action(in make_payoff_row)
    payoff_row = make_payoff_row(op1,v2)  # the vector list out 2*n payoffs after min's action combine with 2*n possible max's actions
    #print(payoff_row)

    # Replace the the column_index of agent v2 with 100 for min
    j_1 = 2*v2 + 0
    j_2 = 2*v2 + 1
    payoff_row[j_1] = 100
    payoff_row[j_2] = 100
    
    print('Min Payoff Row')
    print(payoff_row)
    #calculate fictitious payoff - equi_min  
    payoff_cal = payoff_row * fla_max_fre # fla_max_fre recorded the frequency of each maximizer's action, frequency sum = 1
                                             # payoff (2*n array) * maximizer_action_frequency (2*n array)

    mixed_pol = np.sum(payoff_cal) # add up all, calculate average/expected payoff


#     print('min_mixed_polarization')
#     print(mixed_pol)
        # Replace the the column_index of agent v2 with 100 for min

    payoff_row[j_1] = -100
    payoff_row[j_2] = -100

    return (mixed_pol,payoff_row)


In [751]:
def derivate_s(op,n,v2):
               #op - opinion array that updated by maximizer
    c = [1/n] * n
#     print(c)
    sum_term = 0
    j = 0

    sum_term = np.dot(np.dot((A-c),(A[v2]-c)),op)  # sum up all terms
    
    term_out = op[v2]*np.dot((A[v2]-c),(A[v2]-c)) # exclude the term that j = v2
    sum_s = sum_term - term_out    # numerator
    
    s_star = -sum_s/np.dot((A[v2]-c),(A[v2]-c))
    s_star = s_star[0] #take value out of array
    min_opinion =min(max(0,s_star),1)
            
    return min_opinion

def derivate_s1(op,n,v2):
               #op - opinion array that updated by maximizer
    c = [1/n] * n
#     print(c)
    sum_term = 0
    j = 0
    for j in range(0,n):
        term = op[j]*np.dot(np.transpose(A[j]-c),(A[v2]-c))
#             print(A[j])
#             print(A[v])
        sum_term = sum_term + term  # sum up all terms
    
    term_out = op[v2]*np.dot(np.transpose(A[v2]-c),(A[v2]-c)) # exclude the term that j = v2
    sum_s = sum_term - term_out    # numerator
    
    s_star = -sum_s/np.dot(np.transpose(A[v2]-c),(A[v2]-c))
    s_star = s_star[0] #take value out of array
    min_opinion =min(max(0,s_star),1)
            
    return min_opinion



def min_mixed_opinion(op, n, v2, fla_max_fre):
    
    weight_op = 0
    
    # loop for each max_action(in total 2*n) 
    for column in range(2*n):

        if fla_max_fre[column] !=0:
            v1 = int(column/2)  #i.e., column 11 is agent 5, opinion 1
            max_opinion = column%2
            
##             temp = op[v1,0] 
          
##             op[v1,0]= max_opinion #update innate opinion array with max_action    

            min_opinion = derivate_s(op, n, v2)# find min_s_star for each max_action
#             print(fla_max_fre[column],min_opinion)
            weight_op = weight_op + fla_max_fre[column]*min_opinion # sum up p_i*s_i
#     print(weight_op)
##             op[v1,0] = temp 
    
    (mixed_por, payoff_row) = mixed_min_polarization(s, v2, weight_op,fla_max_fre)
    
    return(weight_op,payoff_row,mixed_por)  

In [752]:
op2=op
op2[0,0]=1
print(op2)
min_opinion1 = derivate_s(op2,n,1)
print(min_opinion1)
min_opinion2 = derivate_s1(op2,n,1)
print(min_opinion2)

[[1. ]
 [0.5]
 [0. ]]
0.49999999999999983
0.49999999999999983


In [753]:
# print(v2,fla_max_fre)
# (weight_op_1,payoff_row,min_por) = min_mixed_opinion_1(s, n, v2, fla_max_fre)
# print(weight_op_1)

In [754]:
# fla_max_fre = [0, 0, 0, 0, 0.65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.35, 0, 0, 0]
# # print(fla_max_fre[13])
# v2 = 1
# (weight_op,payoff_row,min_por) = min_mixed_opinion(s, n, v2, fla_max_fre)
# print(weight_op)

In [755]:
# fla_max_fre = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
# # print(fla_max_fre[13])
# v2 = 1
# (weight_op,payoff_row,min_por) = min_mixed_opinion(s, n, v2, fla_max_fre)

In [756]:
all = list(range(n))    # for all agent 
C1 = [x for x in all if x not in max_touched]  
print(C1)

[0, 2]


In [757]:
# Minimizer search: Go through each agent 

def mixed_choose_min_vertex(s, n, v1, max_opinion, max_touched, fla_max_fre):
    # current polarization that changed by maximizer, "innate" objective that min start with
    op = copy.copy(s)
    op[v1,0] = max_opinion
    print('Check if op has been updated by Maximizer')
    print(op)
    min_por = obj_polarization(A, L, op, n) #min_por- set a standard to compare with pol after min's action
    maxup_por = min_por # store innate max updated polarization
#     print('check maxup por')
#     print(maxup_por)
#     payoffs = []    # create an empty list to store all polarizations   
    champion = (None, None, 0, None)  # assume the best action is champion

    all = list(range(n))    # for all agent 
    C1 = [x for x in all if x not in max_touched]  # for the vertice that Maximizer has not touched
    
    for v2 in C1:   
        print('Min start with agent '+ str(v2) )
        (changed_opinion, payoff_row, por) = min_mixed_opinion(op, n, v2, fla_max_fre)   # find the best new_op option           
        print('changed opinion, por, min_por')
        print(changed_opinion, por, maxup_por)

        if por < min_por:  # if the recent polarization is smaller than the minimum polarization in the history
            min_por = por
                                 # update the recent option as champion
            champion = (v2, changed_opinion, payoff_row, min_por)  
        else:
            print('Innate polarization is smaller than Min action')

    return (champion)  # find the best minimizer's action after going through every new_op option of every agent


In [758]:
# fla_max_fre = [0, 0, 0, 0, 0.51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.49, 0, 0, 0]
# v1 = 5
# max_opinion = 0
# max_touched = []
print('v1,max_opinion')
print(v1,max_opinion)
champion = mixed_choose_min_vertex(s, n, v1, max_opinion, max_touched, fla_max_fre)
# print(champion)

v1,max_opinion
1 0
Check if op has been updated by Maximizer
[[1.]
 [0.]
 [0.]]
Min start with agent 0
Min Payoff Row
[100.    100.      0.      0.593   0.148   0.445]
changed opinion, por, min_por
0.0 0.2962951111158519 0.5933309600094931
Min start with agent 2
Min Payoff Row
[  0.148   0.148   0.445   0.148 100.    100.   ]
changed opinion, por, min_por
0.5 0.296850664449194 0.5933309600094931
Innate polarization is smaller than Min action


In [759]:
####Op has been updated by maximizer, fla_max_fre includes max's hisotry, so minimizer react to the innate op after that
def mixed_min_play(s,v1,max_opinion,n, max_touched,fla_max_fre): 

    print('_______________________')
    print('Minimizer Play')
#     print('Only 1 opinion changed')
#     print(op)
    
    min_champion = mixed_choose_min_vertex(s, n, v1, max_opinion, max_touched, fla_max_fre)
    (v2, min_opinion, payoff_row, min_pol) = min_champion
    
    if v2 == None:    # if minimizer cannot find a action to minimize polarization after maximizer's action
        print('Minimizer fail')

    else:
        print("                                ")
        print("Minimizer finds its target agent:")
#         print('v2', 'changed_opinion', 'innate_obj', 'obj')
#         print(v2, min_opinion, innat_equi_por, min_pol)

        # Store innate_op of the min_selected vertex
        old_opinion_min = op[v2,0]

        print("    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))
#         print("Payoff row")
#         print(payoff_row)
#         print("Network reaches equilibrium Polarization: " + str(min_pol))
#         print('2 opinion changed')
#         print(op)

    return (v2, payoff_row, min_opinion, min_pol)

In [760]:
print(max_touched)
(v2, payoff_row, min_opinion, polarization) = mixed_min_play(s,v1,max_opinion,n, max_touched,fla_max_fre)
print('v2, payoff_row, min_opinion, polarization')


[1]
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1.]
 [0.]
 [0.]]
Min start with agent 0
Min Payoff Row
[100.    100.      0.      0.593   0.148   0.445]
changed opinion, por, min_por
0.0 0.2962951111158519 0.5933309600094931
Min start with agent 2
Min Payoff Row
[  0.148   0.148   0.445   0.148 100.    100.   ]
changed opinion, por, min_por
0.5 0.296850664449194 0.5933309600094931
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent0 's opinion 1.0 changed to 0.0
v2, payoff_row, min_opinion, polarization


### Maximizer Mixed Strategy

In [191]:
# payoff_matrix = np.empty((0, 2*n), float)
# payoff_matrix = np.vstack([payoff_matrix, payoff_row])
# print(payoff_matrix)

In [192]:
####Op has been updated by minimizer, fla_min_fre includes min's hisotry, so maxmizer react to the innate op after that
def mixed_max_polarization(payoff_matrix,v1,max_opinion,fla_min_fre):

    # create payoff matrix for maxmizer
    column = int(column_index(v1,max_opinion))
#     print(payoff_matrix)
#     print("column"+str(column))
    payoff_vector = payoff_matrix[:,column]
    
#     print('payoff vector')
#     print(payoff_vector)

    #calculate fictitious payoff - equi_max   
    payoff_cal = payoff_vector * fla_min_fre #payoff * frequency
    
#     print('max_payoff_calculation')
#     print(payoff_cal)
    mixed_pol = np.sum(payoff_cal) # add up
#     print("Max_mixed_polarization")
#     print(mixed_pol)

    return mixed_pol


In [695]:
# mixed_pol = mixed_max_polarization(payoff_matrix,v1,max_opinion, fla_min_fre)

In [194]:
# determines if value of opinion at v should be set to 0 or 1 to maximize equilibrium polarization 
def max_mixed_opinion(payoff_matrix, n, v1, fla_min_fre):
    
    por_arr = np.zeros(2)  # create a two_element array to store polarization value of each option


    max_opi_option = [0, 1.0]   # Maximizer has two options to change agent v1's opinion
    
    # objective if set opinion to 0, 1.0
    j = 0
    for new_op in max_opi_option:
#         print('change op to '+ str(i/10))
        max_opinion = new_op

        por_arr[j] = mixed_max_polarization(payoff_matrix,v1,max_opinion, fla_min_fre)
    
        j = j + 1   # index increase 1, put the polarization in array

#     print('Polarization Options')
#     print(por_arr)
    
    maxmize_op = np.argmax(por_arr)  # the index of maximum polarization = max_opinion --[0,1]
    max_por = np.max(por_arr)        # find the maximum polarization in the record
 
#     print('new_op', 'innat_equi_por', 'max_por')
#     print(maxmize_op, innat_equi_por, max_por)

    return (maxmize_op, max_por)

In [195]:
# fla_min_fre = [0, 0, 0, 0, 0.65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.35, 0, 0, 0]
# v1 = 2
# champion = max_mixed_opinion(payoff_matrix, n, v1, v2, fla_min_fre)
# print(champion)

In [196]:
# determine which agent maximizer should select to maximizer the equilibrium polarization
def mixed_choose_max_vertex(payoff_matrix,op, n, min_touched, fla_min_fre):
#     print('Check if op has been updated by minimizer')
#     print(op)
    max_por = obj_polarization(A, L, op, n)  # use "innate"(after min action) polarization as a comparable standard to find max_por
    minup_por = max_por # store innate min_update polarization
#     print('check minup por')
#     print(minup_por)
    champion = (None, None, max_por)  # assume champion is the best action

    all = list(range(n))    # for all agent 
    C1 = [x for x in all if x not in min_touched]  # for the vertice that Minimizer has not touched
    for v1 in C1:  
            print('Maximizer start from agent'+str(v1))
            (changed_opinion, por) = max_mixed_opinion(payoff_matrix, n, v1, fla_min_fre)
            print('changed_opinion, por, minup_por')
            print(changed_opinion, por,minup_por)
            
            if por > max_por: # if the polarization of most recent action > maximum polarization of previous actions
                max_por = por
                champion = (v1, changed_opinion,max_por)   # save the this action as champion    
#             else:
#                 print('Innate polarization is bigger than max action')
 
    return (champion)


In [762]:
print(payoff_matrix)
champion = mixed_choose_max_vertex(payoff_matrix,op, n, min_touched, fla_min_fre)

[[0.    0.    0.    0.593 0.148 0.445]
 [0.445 0.148 0.593 0.    0.    0.   ]
 [0.148 0.148 0.445 0.148 0.    0.   ]
 [0.    0.    0.148 0.445 0.148 0.148]]
Maximizer start from agent1
changed_opinion, por, minup_por
1 0.2966654800047467 0.4449982200071198
Maximizer start from agent2
changed_opinion, por, minup_por
1 0.1485179244468207 0.4449982200071198


In [198]:
# # min_touched = []
# # payoff_matrix = np.empty((0, 2*n), float)
# # fla_min_fre = np.empty((0,n))
# # champion = mixed_choose_max_vertex(payoff_matrix,op, n, min_touched, fla_min_fre)
# # print(champion)
# print(c1)
# vertices = np.where(c1)
# print(vertices)
# por=0
# for i in c1:
#     print(i)
#     max_por = 0.75
#     if por > max_por:
#         max_por = por
#         print('yes')
#     else:
#         print('por<max')

In [199]:
def mixed_max_play(payoff_matrix,s,v2,min_opinion,n,min_touched,fla_min_fre): 
    op = copy.copy(s)   # op is a copy of innate opinion
    
    #update innat opinion 
    op[v2,0] = min_opinion  # Op has been updated by minimizer, so maximizer react to the innate op after that
    

    max_champion = mixed_choose_max_vertex(payoff_matrix,op, n, min_touched, fla_min_fre) # The best choice among all opinions and vertexs
    (v1, max_opinion, max_pol) = max_champion

    if v1 == None:
        print('Maximizer fail')

    else:
        print("                                ")
        print("Maximizer finds its target agent:")
        #Store innate_op of the max_selected vertex
        old_opinion_max = op[v1, 0]
        
        ## check if agent's opinionis is changed or not
        print("    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
#         print("Network reaches equilibrium Polarization: " + str(max_pol))
#         print('2 opinion changed')
#         print(op)

    return(v1, max_opinion, max_pol)

In [200]:
# # # Testing function -- NO NEED TO RUN
# min_touched = []
# v2 = 0
# min_opinion = 0
# b = mixed_max_play(payoff_matrix,s,v2,min_opinion,n,min_touched,fla_min_fre)
# print('v1,max_opinion,max_pol')
# print(b)

### Testing Player's Behavior

# 5. Innate Op and Game

# Fictitious Play Start !


In [42]:
# s = make_innat_opinions(n, c1)
# op = s
# print(s)
# y = mean_center(s,n)
# # print(y)
# innat_pol = np.dot(np.transpose(y), y)[0,0] 
# print('Innate_polarization:')
# print(innat_pol)

# # Test equilibrium polarization
# equ_pol = obj_polarization(A, L, op, n)
# print('Equi_polarization:')
# print(equ_pol)

# di = equ_pol-innat_pol
# print("Difference:")
# print(di)



In [673]:
Network = 'Ex_triangle_equal'
# memeory = 50


with open('Network_'+str(Network)+'.txt', "a") as fi:
    print('Innate Opinion', file=fi)
    print(s, file=fi)
    print('Adjacency Matrix', file=fi)
    print(G,file=fi)

# Game Preparation
def push(obj, element):
    if len(obj) >= memory:
        obj.pop(0)
        print('pop')
    obj.append(element)
    return obj


In [601]:
# s = make_innat_opinions(n, c1)
# n = 545
# op = s
# print(s)
n = 3
s_1=[1]
s_2=[0.5]
s_3=[0]
s = np.vstack((s_1,s_2,s_3))
print(s)
#s = (1,0.5,0) # polarization = 0.75
# s = (1,0,0.5) # polarization = 0.75
# s =(0,1,0.5) # polarization = 0.75
#s = (0.5,1,0) # polarization = 0
op = s
y = mean_center(s,n)
# print(y)
innat_pol = np.dot(np.transpose(y), y)[0,0] 
print('Innate_polarization:')
print(innat_pol)

# Test equilibrium polarization
equ_pol = obj_polarization(A, L, op, n)
print('Equi_polarization:')
print(equ_pol)

di = equ_pol-innat_pol
print("Difference:")
print(di)

[[1. ]
 [0.5]
 [0. ]]
Innate_polarization:
0.5
Equi_polarization:
0.45389476266008816
Difference:
-0.04610523733991184


In [602]:
op = copy.copy(s)
print(n)
print(payoff_matrix)

3
[[0.4623 0.1541 0.6164 0.     0.     0.    ]
 [0.1541 0.1541 0.4623 0.1541 0.     0.    ]]


In [1018]:
# Game Parameters
Game_rounds =201 # Rounds + 1- use for printing data
memory = 1

def all_fre_limited_touch(s, n):
    # Preparation for the game
    op = copy.copy(s)
    payoff_matrix = np.empty((0, 2*n), float)
    max_history = np.zeros([n, 2])  # n*2 matrix, agent i & opinion options
    min_history = []  # append a list of (agent i, min_opinion), min_opinion can be any value
    print(type(min_history))

    max_history_last_100 = np.zeros([n, 2]) 
    min_history_last_100= []

    max_touched = []
    min_touched = []
    min_touched_all = []
    min_touched_last_100 =[]
    print('min_touched')
    print(min_touched)
    
    
    # Game start from maximizer random play
    print('Maximizer first selection')
    (v1, max_opinion, max_pol) = random_play(op,n)   # Maximizer does random action 
    #(v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)
    #(v1, max_opinion, max_pol) = (2, 1, 0.14833274000237331)
    First_max = (v1, max_opinion, max_pol) 


#     (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,max_touched)

    # Maximizer start with greedy play
    # (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)   # Maximizer choose action greedily
    max_touched.append(v1)    # save Maximizer's action history

    # store maximizer play history, using agent(row) and changed opinion(column) as indicator to locate history
    max_history[v1,int(max_opinion)] = max_history[v1,int(max_opinion)] +1
    # print('max_history')
    # print(max_history)
    print('history at spot')
    print(max_history[v1,int(max_opinion)])

    max_frequency = max_history/1  # its frequency, only played  1 time so far, divided by 1 
    # print('fre_max at spot')
    # print(max_frequency[v1,int(max_opinion)])

    fla_max_fre = max_frequency.flatten()   # flatten the n*2 matrix to a 2n*1 matrix
                                            # so we can multiply the freuency (2n*1)with payoff array (1*2n) 
                                            # to get average payoff of fictitious play
    print('fre_max at spot')
    print(fla_max_fre)

    column = int(column_index(v1,max_opinion))    # the frequency of maximizer's most recent action (v1,max_opinion)

    print(fla_max_fre[column])

    # print(np.shape(fla_max_fre.shape))


    # if game start from minimizer random play - make sure two random play are not same agent!!!
    print('Minimizer first selection')
    (v2, min_opinion, min_pol) = random_play1(op,n) 
    #(v2, min_opinion, min_pol) = minimizer_fir_play(s,n,min_touched)
    
    #(v2, min_opinion, min_pol) = (1, 0, 0.5933309600094931)
    First_min = (v2, min_opinion, min_pol)

    if v1==v2:   # if Max and Min randomly selected the same agent, then we need to restart - cannot choose same agent
        sys.exit()

    # Minimizer start with greedy play
    # (v2, min_opinion, min_pol) = minimizer_fir_play(s,n,max_touched)

    min_touched.append(v2)
   

    # store minimizer play history
    min_history.append((v2,min_opinion))
    print('min_history')
    print(min_history)


    counter=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
    print(counter)
    fla_min_fre = np.array(list(counter.values()))/1 #return only frequency of all min options in order
    print('fla_min_fre')
    print(fla_min_fre)


    (a,payoff_row) = mixed_min_polarization(s,v2,min_opinion,fla_max_fre)
    payoff_matrix = np.vstack([payoff_matrix, payoff_row])
    print('Payoff Matrix')
    print(payoff_matrix)
    # print('fla_min_fre at the spot')
    # min_counter = dict(counter)
    # print(min_counter) 
    # print(min_counter[(v2,min_opinion)]) 
    # print(min_counter[(v2,min_opinion)]/(i+1)) #get the value from dictionary by using key (v2,opinion)


    equi_min = min_pol
    equi_max = max_pol
    # print(equi_min)
    # print(equi_max)



    Flag = 0

    i = 0
    while Flag == 0: 
        i = i + 1
        print("Game " + str(i))
        print("_____________________")

    #     if max_pol == min_pol:
        if i == Game_rounds:            # i == # of iterations you want to run + 2
                                # because Game 101 is skipped for collecting data, to get 200 game result, we need to run 201 iteration
            print('min_recent_'+str(memory)+'_touched')# then stop at Game 202
            print(min_touched)
            print('max_recent_'+str(memory)+'_touched')
            print(max_touched)
            print('Min last 100 action')
            print(min_touched_last_100)

            break

        elif equi_min == equi_max:
            print("Reached Nash Equilibrium at game"+ str(i) + "and Equi_Por = " + str(equi_min))
            print('max_distribution')
            print(max_frequency)
            print('min_distribution')
            print(fla_min_fre)
            Flag = 1
            break
        ############################## maximizer play  
        else:
            if i == Game_rounds-100:    #if Game_round = 200, after 100 iteration, Game 101 print previous historical result
    #             max_touched_100 = max_touched 
    #             min_touched_100 = min_touched
    #             max_fre_100 = max_frequency  # store the max_frequency of first 100 iterataions
    #             print('max_history')
    #             print(max_history)
    #             min_fre_100 = fla_min_fre  # max_frequency of first 100 iterations
    #             print('min_history')
    #             print(min_history)
    # Remove max frequncy less than 0.1--
                max_history_last_100 = np.zeros([n, 2]) 
                min_history_last_100 = [] 
                min_touched_last_100 =[]

            (v1, max_opinion, equi_max) = mixed_max_play(payoff_matrix,s,v2,min_opinion,n,min_touched,fla_min_fre)
            max_touched = push(max_touched, v1)
    #         print('min_touched')
    #         print(min_touched)
    #         print('max_touched')
    #         print(max_touched)
    #             print('equi_max')
    #             print(equi_max)
    #         print(v1, max_opinion, max_pol)
            # cumulate strategy 
            max_history[v1,int(max_opinion)] = max_history[v1,int(max_opinion)] +1

            max_history_last_100[v1,int(max_opinion)] = max_history_last_100[v1,int(max_opinion)] +1
    #         print('max_history')
    #         print(max_history)
    #________________________________________________________________
            max_frequency = max_history/(i+1)  # its frequency 
    #         print('max_distribution')
    #         print(max_frequency)
        #     print(i+1) 
            fla_max_fre = max_frequency.flatten() #flaten max_frequency to calculate average payoff
            print('fla_max_fre')
            print(fla_max_fre)
            print('fre_max at spot')
            print(fla_max_fre[column])
            # create payoff matrix for maxmizer
            row = int(row_index(v2, min_opinion))
            column = int(column_index(v1,max_opinion))

    # _________________________________________________________________
    #         ######################Visualize Maximizer's selection
    #         La = scipy.sparse.csgraph.laplacian(G, normed=False)

    #         nxG = nx.from_numpy_matrix(G)

    #         color_map = []
    #         for node in nxG:
    #             if node == v1:
    #                 color_map.append('Red')
    #             else: 
    #                 color_map.append('Grey')  

    #         #nxG1 = nx.DiGraph(G)
    #         nx.draw(nxG, node_color=color_map, with_labels=True,node_size = 50)
    #         plt.figure(figsize=(200, 200))
    #         plt.show()



    ############################### minimizer play
            (v2, payoff_row, min_opinion, equi_min) = mixed_min_play(s,v1,max_opinion,n, max_touched,fla_max_fre)
            min_touched = push(min_touched, v2)
            min_touched_all.append(v2) 
            min_touched_last_100.append(v2)
    #         print('min_touched')
    #         print(min_touched)
    #         print('equi_min')
    #         print(equi_min)
    #         print('max_touched')
    #         print(max_touched)
            #         print(v2, min_opinion, min_pol)
            if (v2,min_opinion) in counter.keys():
                payoff_matrix = payoff_matrix # if this min_option is in min_history, no need to update paryoff matrix, only update frequency
                print("Same history")
                print((str(v2),str(min_opinion)))
            else:
                payoff_matrix = np.vstack([payoff_matrix, payoff_row]) # if this is a new option, append to previous matrix
    #                 print('payoff_row')
    #                 print(payoff_row)
            min_history.append((v2,min_opinion))
            min_history_last_100.append((v2,min_opinion))
            #         print('min_history')
            #         print(min_history)
            counter=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
            #print(counter)
    #         print('counter.keys')
    #         print(counter.keys())
            fla_min_fre = np.array(list(counter.values()))/(i+1) #return only frequency of all min options in order
    #         print('fla_min_fre')
    #         print(fla_min_fre)

    #         print('fla_min_fre at the spot')
    #         min_counter = dict(counter)
    #         print(min_counter[(v2,min_opinion)]/(i+1)) #get the value from dictionary by using key (v2,opinion)

            # create payoff matrix for minimizer
            row = row_index(v2, min_opinion)
            column = column_index(v1,max_opinion)
            #     print('row, column')
            #     print(row, column)

            print("Not Reached Nash Equilibrium at Equi_Min = " + str(equi_min) + " and Equi_Max = "+ str(equi_max)) 
    #         print('min_distribution')
    #         print(fla_min_fre)

            ######################Visualize Minimizer selection
    #         La = scipy.sparse.csgraph.laplacian(G1, normed=False)

    #         nxG = nx.from_numpy_matrix(G1)

    #         color_map = []
    #         for node in nxG:
    #             if node == v2:
    #                 color_map.append('Blue')
    #             else: 
    #                 color_map.append('Grey')  

    #         #nxG1 = nx.DiGraph(G)
    #         nx.draw(nxG, node_color=color_map, with_labels=True)
    #         plt.figure(figsize=(25, 25))
    #         plt.show()
    return (First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100, min_touched_last_100, min_touched_all, max_history, max_history_last_100)



In [1051]:
(First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100, min_touched_last_100, min_touched_all, max_history, max_history_last_100) = all_fre_limited_touch(s, n)


<class 'list'>
min_touched
[]
Maximizer first selection
    Agent2 's opinion 0.0 changed to 1
Network reaches equilibrium Polarization: 0.14833274000237331
history at spot
1.0
fre_max at spot
[0. 0. 0. 0. 0. 1.]
1.0
Minimizer first selection
    Agent1 's opinion 0.5 changed to 0
Network reaches equilibrium Polarization: 0.5933309600094931
min_history
[(1, 0)]
Counter({(1, 0): 1})
fla_min_fre
[1.]
Min Payoff Row
[  0.       0.5933 100.     100.       0.5933   0.5933]
Payoff Matrix
[[   0.        0.5933 -100.     -100.        0.5933    0.5933]]
Game 1
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.5933309600094931 0.5933309600094931
Maximizer start from agent2
changed_opinion, por, minup_por
1 0.5933309600094933 0.5933309600094931
                                
Maximizer finds its target agent:
    Agent2 's opinion 0.0 changed to 1
pop
fla_max_fre
[0. 0. 0. 0. 0. 1.]
fre_max at spot
1.0
_______________________
Minimizer Play
Check if op has bee

    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9231 0.     0.     0.     0.0769]
fre_max at spot
0.9230769230769231
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0. ]]
Min start with agent 1
Min Payoff Row
[  0.1483   0.445  100.     100.       0.445    0.1483]
changed opinion, por, min_por
0.49999999999999983 0.4221777984682931 0.4449982200071198
Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 7.794999589232413 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4221777984682931 and Equi_Max = 0.4568648392073096
Game 26
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por


[  0.1483   0.445  100.     100.       0.445    0.1483]
changed opinion, por, min_por
0.49999999999999983 0.4318130875624644 0.4449982200071198
Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 4.550749574779479 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4318130875624644 and Equi_Max = 0.4517406172799549
Game 45
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.4515907862294475 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.4515907862294475 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9

Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 3.006575654856801 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4363992205866923 and Equi_Max = 0.4493609476542484
Game 69
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.4492977197173335 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.4492977197173335 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9714 0.     0.     0.     0.0286]
fre_max at spot
0.9714285714285714
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0.

Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 2.4615730948840913 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4380178557717139 and Equi_Max = 0.4485299519119382
Game 85
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.44848840212482266 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.44848840212482266 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9767 0.     0.     0.     0.0233]
fre_max at spot
0.9767441860465116
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 

Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.7499999999999999 2.032187063559438 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4392931146224131 and Equi_Max = 0.4478784673858066
Game 104
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.4478507726994731 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.4478507726994731 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.    0.981 0.    0.    0.    0.019]
fre_max at spot
0.9809523809523809
_______________________
Minimizer Play
Check if op has been updated

[0.    0.984 0.    0.    0.    0.016]
fre_max at spot
0.984
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0. ]]
Min start with agent 1
Min Payoff Row
[  0.1483   0.445  100.     100.       0.445    0.1483]
changed opinion, por, min_por
0.49999999999999983 0.4402515723270438 0.4449982200071198
Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 1.7094695621217515 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.4402515723270438 and Equi_Max = 0.4473906835555451
Game 125
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.4473715438471577 0.44499822000711975
Maximizer start from agent2
cha

Not Reached Nash Equilibrium at Equi_Min = 0.4410161330271903 and Equi_Max = 0.44700271649363826
Game 149
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.44698926349708445 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.44698926349708445 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9867 0.     0.     0.     0.0133]
fre_max at spot
0.9866666666666667
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0. ]]
Min start with agent 1
Min Payoff Row
[  0.1483   0.445  100.     100.       0.445    0.1483]
changed opinion, por, min_por
0.49999999999999983 0.4410426802737231 0.4449982200071198
Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 1.4430995609350896 0.4449982200071198
Inn

Maximizer start from agent2
changed_opinion, por, minup_por
0 0.4467853614529315 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.    0.988 0.    0.    0.    0.012]
fre_max at spot
0.9880239520958084
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0. ]]
Min start with agent 1
Min Payoff Row
[  0.1483   0.445  100.     100.       0.445    0.1483]
changed opinion, por, min_por
0.49999999999999983 0.4414453400070629 0.4449982200071198
Min start with agent 2
Min Payoff Row
[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 1.3075220154209204 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equili

[2.5958e-01 1.1125e-01 4.8208e-01 3.7083e-02 1.0000e+02 1.0000e+02]
changed opinion, por, min_por
0.75 1.1853221404318686 0.4449982200071198
Innate polarization is smaller than Min action
                                
Minimizer finds its target agent:
    Agent1 's opinion 0.5 changed to 0.49999999999999983
pop
Same history
('1', '0.49999999999999983')
Not Reached Nash Equilibrium at Equi_Min = 0.44180826860921923 and Equi_Max = 0.44660181719633457
Game 186
_____________________
Maximizer start from agent0
changed_opinion, por, minup_por
1 0.44659319570607003 0.44499822000711975
Maximizer start from agent2
changed_opinion, por, minup_por
0 0.44659319570607003 0.44499822000711975
                                
Maximizer finds its target agent:
    Agent0 's opinion 1.0 changed to 1
pop
fla_max_fre
[0.     0.9893 0.     0.     0.     0.0107]
fre_max at spot
0.9893048128342246
_______________________
Minimizer Play
Check if op has been updated by Maximizer
[[1. ]
 [0.5]
 [0. ]]
Min s

In [1052]:
print(fla_min_fre)
print(min_touched)
print(min_history_last_100)

[0.005  0.005  0.1343 0.8458 0.01  ]
[1]
[(1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.4999999999999998), (1, 0.49999999999999983), (1, 0.4999999999999998), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.4999999999999998), (1, 0.49999999999999983), (1, 0.4999999999999998), (1, 0.4999999999999998), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.4999999999999998), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1, 0.49999999999999983), (1

In [1053]:
print(payoff_matrix)
print(payoff_row)
# print(First_max)
# print(First_min)
print(G)

[[ 0.0000e+00  5.9333e-01 -1.0000e+02 -1.0000e+02  5.9333e-01  5.9333e-01]
 [ 5.9333e-01  5.9333e-01 -1.0000e+02 -1.0000e+02  5.9333e-01  6.5820e-32]
 [ 1.4833e-01  4.4500e-01 -1.0000e+02 -1.0000e+02  4.4500e-01  1.4833e-01]
 [ 1.4833e-01  4.4500e-01 -1.0000e+02 -1.0000e+02  4.4500e-01  1.4833e-01]
 [ 1.4833e-01  4.4500e-01 -1.0000e+02 -1.0000e+02  4.4500e-01  1.4833e-01]]
[   0.445     0.1483    0.5933    0.     -100.     -100.    ]
[[0.   0.02 0.02]
 [0.02 0.   0.02]
 [0.02 0.02 0.  ]]


In [1048]:
print(s)

[[1. ]
 [0.5]
 [0. ]]


In [1049]:
# Experiment = 0

# Experiment_note = str('Note: This experiement has initial condition. Game round:'+str(Game_rounds)+'.')
# (First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100, min_touched_last_100, min_touched_all, max_history, max_history_last_100) = all_fre_limited_touch(s, n)



In [1050]:
# MAXimizer's distribution of LAST 100 iteration 
print('Max_distribution')  
max_l100_fre = max_history_last_100/100
print(max_l100_fre [np.nonzero(max_l100_fre)])
# print for small network
print(max_history_last_100)
# # Print for Large Network
# print(np.nonzero(max_l100_fre))

# MINimizer's Strategy in the last 100 round
print('Min_distribution')
counter=collections.Counter(min_history_last_100)  #return a dictionary include {'min_option': count of this choice}
print(counter.keys())
fla_min_fre = np.array(list(counter.values()))/(100) #return only frequency of all min options in order
print('fla_min_fre')
print(fla_min_fre)

print(min_touched_last_100)
counter_1=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
print(counter_1)
fla_min_fre_1 = np.array(list(counter_1.values()))/Game_rounds #return only frequency of all min options in order
print('fla_min_fre_1')
print(fla_min_fre_1)


Max_distribution
[1.]
[[  0. 100.]
 [  0.   0.]
 [  0.   0.]]
Min_distribution
dict_keys([(1, 0.49999999999999983), (1, 0.4999999999999998)])
fla_min_fre
[0.92 0.08]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Counter({(1, 0.49999999999999983): 170, (1, 0.4999999999999998): 27, (1, 0.4999999999999999): 2, (1, 0): 1, (1, 0.9999999999999997): 1})
fla_min_fre_1
[0.005  0.005  0.1343 0.8458 0.01  ]


In [687]:
print(payoff_matrix)

[[0.    0.    0.    0.593 0.148 0.445]
 [0.445 0.148 0.593 0.    0.    0.   ]
 [0.148 0.148 0.445 0.148 0.    0.   ]
 [0.    0.    0.148 0.445 0.148 0.148]] 1


In [674]:
Experiment = 'Pure11'
with open('Result'+str(Network)+'.'+str(Experiment)+'.txt', "a") as f:
#     print(Experiment_note, file=f)
    print('Initial Condition -(agent, opinion, pol)', file=f)
    print('Innate op'+str(s),file=f)
    print('Adjacency matrix'+ str(G), file=f)
    print('Max:'+ str(First_max), file=f)
    print('Min' + str(First_min), file=f)

    print("In the Last 100 Rounds", file=f) 
    print('_____________________', file=f)
    
    # MAX distribution of LAST 100 iteration 
    print('Max_distribution', file=f)  
    max_l100_fre = max_history_last_100/100
    print(max_l100_fre [np.nonzero(max_l100_fre)], file=f)
    # print for small network
    print(max_history_last_100, file=f)
    # # Print for Large Network
    # print(np.nonzero(max_l100_fre))

    # MIN Strategy in the last 100 round
    counter=collections.Counter(min_history_last_100)  #return a dictionary include {'min_option': count of this choice}
    # print(counter)
    fla_min_fre = np.array(list(counter.values()))/100 #return only frequency of all min options in order
    print('Min_frequency', file=f)
    print(list(counter.keys()), file=f)
    print('Min_distribution_last_100', file=f)
    print(fla_min_fre, file=f)
    counter_h=collections.Counter(min_history_last_100)  #return a dictionary include {'min_option': count of this choice}
    print(counter_h.keys(), file=f)
    
    print('min_recent_'+str(memory)+'_touched', file=f)# then stop at Game 202
    print(min_touched, file=f)
    print('max_recent_'+str(memory)+'_touched', file=f)
    print(max_touched, file=f)
    
    print('In Overall'+str(Game_rounds)+' Rounds', file=f)
    print('_____________________', file=f)
    
    # Max action Overall 
    np.set_printoptions(precision=3)

    max_fre = max_history/Game_rounds
    print('Max_frequency', file=f)
    print(max_history, file=f)
    print('Max_distribution', file=f)
    print(max_fre [np.nonzero(max_fre)], file=f)
    


    # Min Strategy in the Overall    
    counter_1=collections.Counter(min_touched_all)  #return a dictionary include {'min_option': count of this choice}
    fla_min_fre_all = np.array(list(counter_1.values()))/Game_rounds #return only frequency of all min options in order
    print('Min_dist_all', file=f)
    print(fla_min_fre_all, file=f)
    print('Min_distribution', file=f)
    counter_a=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
    print(counter_a.keys(), file=f)
    print(payoff_matrix, file=f)

In [670]:
counter=collections.Counter(min_history) 
print(counter)
fla_min_fre = np.array(list(counter.values()))/Game_rounds
print('fla_min_fre')
print(fla_min_fre)

Counter({(1, 0.49999999999999983): 366, (1, 0.4999999999999998): 31, (1, 0.4999999999999999): 2, (1, 0): 1, (1, 0.9999999999999997): 1})
fla_min_fre
[0.002 0.002 0.077 0.913 0.005]


In [845]:
# # Game Parameters
# Game_rounds =801 # Rounds + 1- use for printing data
# memory = 1

# # def all_fre_limited_touch(s, n):
# #     # Preparation for the game
# op = copy.copy(s)
# payoff_matrix = np.empty((0, 2*n), float)
# max_history = np.zeros([n, 2])  # n*2 matrix, agent i & opinion options
# min_history = []  # append a list of (agent i, min_opinion), min_opinion can be any value
# print(type(min_history))

# max_history_last_100 = np.zeros([n, 2]) 
# min_history_last_100= []

# max_touched = []
# min_touched = []
# min_touched_all = []
# min_touched_last_100 =[]
# print('min_touched')
# print(min_touched)


# # Game start from maximizer random play
# print('Maximizer first selection')
# (v1, max_opinion, max_pol) = random_play(op,n)   # Maximizer does random action 
# #(v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)
# # (v1, max_opinion, max_pol) = (1, 1, 0.6163708086785011)
# First_max = (v1, max_opinion, max_pol) 


# #     (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,max_touched)

# # Maximizer start with greedy play
# # (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)   # Maximizer choose action greedily
# max_touched.append(v1)    # save Maximizer's action history

# # store maximizer play history, using agent(row) and changed opinion(column) as indicator to locate history
# max_history[v1,int(max_opinion)] = max_history[v1,int(max_opinion)] +1
# # print('max_history')
# # print(max_history)
# print('history at spot')
# print(max_history[v1,int(max_opinion)])

# max_frequency = max_history/1  # its frequency, only played  1 time so far, divided by 1 
# # print('fre_max at spot')
# # print(max_frequency[v1,int(max_opinion)])

# fla_max_fre = max_frequency.flatten()   # flatten the n*2 matrix to a 2n*1 matrix
#                                         # so we can multiply the freuency (2n*1)with payoff array (1*2n) 
#                                         # to get average payoff of fictitious play
# print('fre_max at spot')
# print(fla_max_fre)

# column = int(column_index(v1,max_opinion))    # the frequency of maximizer's most recent action (v1,max_opinion)

# print(fla_max_fre[column])

# # print(np.shape(fla_max_fre.shape))


# # if game start from minimizer random play - make sure two random play are not same agent!!!
# print('Minimizer first selection')
# (v2, min_opinion, min_pol) = random_play(op,n) 
# #(v2, min_opinion, min_pol) = minimizer_fir_play(s,n,min_touched)

# # (v2, min_opinion, min_pol) = (0, 0, 0.15409270216962534)
# First_min = (v2, min_opinion, min_pol)

# if v1==v2:   # if Max and Min randomly selected the same agent, then we need to restart - cannot choose same agent
#     sys.exit()

# # Minimizer start with greedy play
# # (v2, min_opinion, min_pol) = minimizer_fir_play(s,n,max_touched)

# min_touched.append(v2)


# # store minimizer play history
# min_history.append((v2,min_opinion))
# print('min_history')
# print(min_history)


# counter=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
# print(counter)
# fla_min_fre = np.array(list(counter.values()))/1 #return only frequency of all min options in order
# print('fla_min_fre')
# print(fla_min_fre)


# (a,payoff_row) = mixed_min_polarization(s,v2,min_opinion,fla_max_fre)
# payoff_matrix = np.vstack([payoff_matrix, payoff_row])
# print('Payoff Matrix')
# print(payoff_matrix)
# # print('fla_min_fre at the spot')
# # min_counter = dict(counter)
# # print(min_counter) 
# # print(min_counter[(v2,min_opinion)]) 
# # print(min_counter[(v2,min_opinion)]/(i+1)) #get the value from dictionary by using key (v2,opinion)


# equi_min = min_pol
# equi_max = max_pol
# # print(equi_min)
# # print(equi_max)



# Flag = 0

# i = 0
# while Flag == 0: 
#     i = i + 1
#     print("Game " + str(i))
#     print("_____________________")

# #     if max_pol == min_pol:
#     if i == Game_rounds:            # i == # of iterations you want to run + 2
#                             # because Game 101 is skipped for collecting data, to get 200 game result, we need to run 201 iteration
#         print('min_recent_'+str(memory)+'_touched')# then stop at Game 202
#         print(min_touched)
#         print('max_recent_'+str(memory)+'_touched')
#         print(max_touched)
#         print('Min last 100 action')
#         print(min_touched_last_100)

#         break

#     elif equi_min == equi_max:
#         print("Reached Nash Equilibrium at game"+ str(i) + "and Equi_Por = " + str(equi_min))
#         print('max_distribution')
#         print(max_frequency)
#         print('min_distribution')
#         print(fla_min_fre)
#         Flag = 1
#         break
#     ############################## maximizer play  
#     else:
#         if i == Game_rounds-100:    #if Game_round = 200, after 100 iteration, Game 101 print previous historical result
# #             max_touched_100 = max_touched 
# #             min_touched_100 = min_touched
# #             max_fre_100 = max_frequency  # store the max_frequency of first 100 iterataions
# #             print('max_history')
# #             print(max_history)
# #             min_fre_100 = fla_min_fre  # max_frequency of first 100 iterations
# #             print('min_history')
# #             print(min_history)
# # Remove max frequncy less than 0.1--
#             max_history_last_100 = np.zeros([n, 2]) 
#             min_history_last_100 = [] 
#             min_touched_last_100 =[]
#         print('Max_Payoff_Matrix')
#         print(payoff_matrix)
#         (v1, max_opinion, equi_max) = mixed_max_play(payoff_matrix,s,v2,min_opinion,n,min_touched,fla_min_fre)
#         max_touched = push(max_touched, v1)
# #         print('min_touched')
# #         print(min_touched)
# #         print('max_touched')
# #         print(max_touched)
# #             print('equi_max')
# #             print(equi_max)
# #         print(v1, max_opinion, max_pol)
#         # cumulate strategy 
#         max_history[v1,int(max_opinion)] = max_history[v1,int(max_opinion)] +1

#         max_history_last_100[v1,int(max_opinion)] = max_history_last_100[v1,int(max_opinion)] +1
# #         print('max_history')
# #         print(max_history)
# #________________________________________________________________
#         max_frequency = max_history/(i+1)  # its frequency 
# #         print('max_distribution')
# #         print(max_frequency)
#     #     print(i+1) 
#         fla_max_fre = max_frequency.flatten() #flaten max_frequency to calculate average payoff
#         print('fla_max_fre')
#         print(fla_max_fre)
#         print('fre_max at spot')
#         print(fla_max_fre[column])
#         # create payoff matrix for maxmizer
#         row = int(row_index(v2, min_opinion))
#         column = int(column_index(v1,max_opinion))

# # _________________________________________________________________
# #         ######################Visualize Maximizer's selection
# #         La = scipy.sparse.csgraph.laplacian(G, normed=False)

# #         nxG = nx.from_numpy_matrix(G)

# #         color_map = []
# #         for node in nxG:
# #             if node == v1:
# #                 color_map.append('Red')
# #             else: 
# #                 color_map.append('Grey')  

# #         #nxG1 = nx.DiGraph(G)
# #         nx.draw(nxG, node_color=color_map, with_labels=True,node_size = 50)
# #         plt.figure(figsize=(200, 200))
# #         plt.show()



# ############################### minimizer play
#         (v2, payoff_row, min_opinion, equi_min) = mixed_min_play(s,v1,max_opinion,n, max_touched,fla_max_fre)
# #         print('Min_Payoff_row')
# #         print(payoff_row)
#         min_touched = push(min_touched, v2)
#         min_touched_all.append(v2) 
#         min_touched_last_100.append(v2)
# #         print('min_touched')
# #         print(min_touched)
# #         print('equi_min')
# #         print(equi_min)
# #         print('max_touched')
# #         print(max_touched)
#         #         print(v2, min_opinion, min_pol)
#         if (v2,min_opinion) in counter.keys():
#             payoff_matrix = payoff_matrix # if this min_option is in min_history, no need to update paryoff matrix, only update frequency
#             print("Same history")
#             print((str(v2),str(min_opinion)))
#         else:
#             payoff_matrix = np.vstack([payoff_matrix, payoff_row]) # if this is a new option, append to previous matrix
# #                 print('payoff_row')
# #                 print(payoff_row)
#         min_history.append((v2,min_opinion))
#         min_history_last_100.append((v2,min_opinion))
#         #         print('min_history')
#         #         print(min_history)
#         counter=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
#         #print(counter)
# #         print('counter.keys')
# #         print(counter.keys())
#         fla_min_fre = np.array(list(counter.values()))/(i+1) #return only frequency of all min options in order
# #         print('fla_min_fre')
# #         print(fla_min_fre)

# #         print('fla_min_fre at the spot')
# #         min_counter = dict(counter)
# #         print(min_counter[(v2,min_opinion)]/(i+1)) #get the value from dictionary by using key (v2,opinion)

#         # create payoff matrix for minimizer
#         row = row_index(v2, min_opinion)
#         column = column_index(v1,max_opinion)
#         #     print('row, column')
#         #     print(row, column)

#         print("Not Reached Nash Equilibrium at Equi_Min = " + str(equi_min) + " and Equi_Max = "+ str(equi_max)) 
# #         print('min_distribution')
# #         print(fla_min_fre)

#             ######################Visualize Minimizer selection
#     #         La = scipy.sparse.csgraph.laplacian(G1, normed=False)

#     #         nxG = nx.from_numpy_matrix(G1)

#     #         color_map = []
#     #         for node in nxG:
#     #             if node == v2:
#     #                 color_map.append('Blue')
#     #             else: 
#     #                 color_map.append('Grey')  

#     #         #nxG1 = nx.DiGraph(G)
#     #         nx.draw(nxG, node_color=color_map, with_labels=True)
#     #         plt.figure(figsize=(25, 25))
#     #         plt.show()
#    ## return (First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100, min_touched_last_100, min_touched_all, max_history, max_history_last_100)